## 构建VGG-19模型

In [1]:
import scipy.io 
import numpy as np 
import tensorflow as tf 

#### 1. 层字典

In [2]:
VGG19_LAYERS = {
    'conv1_1','relu1_1','conv1_2','pool1',
    
    'conv2_1','relu2_1','conv2_2','pool2',
    
    'conv3_1','relu3_1','conv3_2','relu3_2','conv3_3','relu3_3','conv3_4','relu3_4','pool3',
    
    'conv4_1','relu4_1','conv4_2','relu4_2','conv4_3','relu4_3','conv4_4','relu4_4','pool4',
    
    'conv5_1','relu5_1','conv5_2','relu5_2','conv5_3','relu5_3','conv5_4','relu5_4'
}

#### 2. 加载模型

In [3]:
#加载模型，并返回权重和平均像素
def load_net(data_path):
    data = scipy.io.loadmat(data_path)
    if 'normalization' in data:
        mean_pixel = np.mean(data['normalization'][0][0][0],axis=(0,1))
    else:
        mean_pixel = data['meta']['normalization'][0][0][0][0][2][0][0]
    weights = data['layers'][0]
    return weights,mean_pixel

#### 3.网络预加载

In [4]:
def net_preloaded(weights, input_image, pooling):
    net = {}
    current = input_image
    for i, name in enumerate(VGG19_LAYERS):
        kind = name[:4]
        if kind == 'conv':
            if isinstance(weights[i][0][0][0][0], np.ndarray):
                # old format
                kernels, bias = weights[i][0][0][0][0]
            else:
                # new format
                kernels, bias = weights[i][0][0][2][0]
            # tensorflow: weights are [height, width, in_channels, out_channels]
            kernels = np.transpose(kernels, (1, 0, 2, 3))
            bias = bias.reshape(-1)
            current = _conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = _pool_layer(current, pooling)
        net[name] = current

    assert len(net) == len(VGG19_LAYERS)
    return net

#### 4.各层构建

In [5]:
def _conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, tf.constant(weights), strides=(1, 1, 1, 1),
            padding='SAME')
    return tf.nn.bias_add(conv, bias)


def _pool_layer(input, pooling):
    if pooling == 'avg':
        return tf.nn.avg_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                padding='SAME')
    else:
        return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                padding='SAME')

#### 5.图片大小处理

In [7]:
def preprocess(image, mean_pixel):
    return image - mean_pixel


def unprocess(image, mean_pixel):
    return image + mean_pixel